In [1]:
import cv2
import numpy as np

In [2]:
cap = cv2.VideoCapture(1)

In [7]:
def play(func=None, vid=cap):
    while True:
        success, frame = vid.read()
        if func:
            frame = func(frame)
        
        cv2.imshow('video', frame)
        if cv2.waitKey(16) & 0XFF==ord('q'):
            break
    cv2.destroyAllWindows()
play()

In [8]:
myColors = [
    [5,107,0,19,255,255],
    [133,56,0,159,256,255],
    [57,76,0,100,255,255],
    [90,48,0,118,255,255]
]
colorValues = [
    (51,153,255),
    (255,0,255),
    (0,255,0),
    (255,0,0)
]

In [9]:
def find_point(mask):
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, w, h = 0, 0, 0, 0
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if not area > 500:
            continue
        peri = cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, 0.02*peri, True)
        x,y,w,h = cv2.boundingRect(approx)
    return (x+w)//2, y


def findColor(img, myColors, colorValues):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    
    newP = []
    for c, color in enumerate(myColors):
        lower = np.array(color[:3])
        upper = np.array(color[3:])

        mask = cv2.inRange(hsv, lower, upper)
        x, y = find_point(mask)
        if x!=0 and y!=0:
            newP.append((x,y,c))
    return newP
        
        
def canvasdraw(img, points, colorValues=colorValues):
    for p in points:
        cv2.circle(img, (p[0], p[1]), 10, colorValues[p[2]], cv2.FILLED)
    return img

def drawColor(img):
    global points
    img = cv2.GaussianBlur(img, (3,3), 1)
    newP = findColor(img, myColors, colorValues)
    points += newP
    
    return canvasdraw(img, points)

In [10]:
points = []
play(drawColor)